In [72]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.llm
import clin.parse
from typing import List
results_dir = '../results/'
from clin.config import PATH_REPO
import clin.eval.ebm
import clin.eval.eval
from clin.modules import ebm
import joblib
import imodelsx.process_results
from IPython.display import HTML
import clin.viz
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 15/15 [00:00<00:00, 1646.07it/s]


In [79]:
# get human spans
df_spans = joblib.load(join(PATH_REPO, 'data', 'ebm', 'ebm_interventions_spans.pkl'))
nums = np.arange(100).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe_spans = df_spans.iloc[nums]

# get predicted evidence
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[r.dataset_name == 'ebm']
row = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[0]
d_evidence = row['dict_evidence_ov_pv_ev']

100%|██████████| 15/15 [00:00<00:00, 1773.89it/s]


In [104]:
def calculate_mean_matches(d_evidence, dfe_spans):
    """Finds mean number of times evidence span from llm contains a token from the human span.
    Mean is taken for each document and then averaged over all documents.
    Baseline is probability that any token falls into a human span.
    """

    # single example
    mean_matches = []
    mean_matches_random = []
    mean_num_tokens = []
    for i in range(len(dfe_spans)):
        span = dfe_spans.iloc[i]
        doc = span['doc'].lower()
        toks = [tok.lower() for tok in span['toks_list']]
        annot = span['annot_list']
        # color_str = clin.viz.colorize(span['toks_list'], span['annot_list'], char_width_max=60, title=str(i) + " " + span['doc_id'])
        # display(HTML(color_str))

        # given reference text and set of tokens, find starting index of each token
        def _find_token_idxs(doc, toks):
            starts = []
            ends = []
            for tok in toks:
                idx = doc.find(tok, ends[-1] if len(ends) > 0 else 0)
                if idx == -1:
                    print('ERROR: token not found:', tok)
                    return None
                starts.append(idx)
                ends.append(idx + len(tok))
            
            # check that idxs are strictly increasing
            for i in range(1, len(starts)):
                if starts[i] <= starts[i - 1]:
                    print('ERROR: idxs not strictly increasing')
                    return None
            return starts, ends
        starts, ends = _find_token_idxs(doc, toks)

        def _get_overlapping_token_idxs(start: int, end: int, starts: List[int], ends: List[int]):
            """
            Given a span [start, end), find the indices of all tokens that overlap with the span.
            """
            idxs = []
            for i in range(len(starts)):
                if start < ends[i] and end > starts[i]:
                    # print(start, end, starts[i], ends[i])
                    idxs.append(i)
            return idxs

        evidence = d_evidence[i]
        matches = []
        num_toks = []
        for intervention in evidence:
            # idx = doc.find(intervention.lower())
            
            # search over all matches
            idxs_match = [m.start() for m in re.finditer(intervention.lower(), doc)]
            for idx_match in idxs_match:
                # print(intervention, doc[idx: idx + len(intervention)])
                tok_idxs = _get_overlapping_token_idxs(idx_match, idx_match + len(intervention), starts, ends)
                # print(np.array(toks)[tok_idxs])
                # print(intervention, tok_idxs, annot[tok_idxs])
                matches.append(np.any(annot[tok_idxs] > 0))
                num_toks.append(len(tok_idxs))
        mean_matches_random.append(np.nanmean(annot > 0))
        mean_matches.append(np.nanmean(matches))
        mean_num_tokens.append(np.nanmean(num_toks))
    return np.nanmean(mean_matches), np.nanmean(mean_num_tokens), np.nanmean(mean_matches_random)

calculate_mean_matches(d_evidence, dfe_spans)

/tmp/ipykernel_1866145/1503278224.py:66: RuntimeWarning: Mean of empty slice
  mean_matches.append(np.nanmean(matches))
/tmp/ipykernel_1866145/1503278224.py:67: RuntimeWarning: Mean of empty slice
  mean_num_tokens.append(np.nanmean(num_toks))


(0.8212752430114761, 2.038433998384656, 0.07789028384988256)

In [112]:
r['Match accuracy'] = r.apply(lambda row: calculate_mean_matches(row['dict_evidence_ov_pv_ev'], dfe_spans)[0], axis=1)
r['Span length'] = r.apply(lambda row: calculate_mean_matches(row['dict_evidence_ov_pv_ev'], dfe_spans)[1], axis=1)
print('random baseline', r.apply(lambda row: calculate_mean_matches(row['dict_evidence_ov_pv_ev'], dfe_spans)[2], axis=1)[0])

/tmp/ipykernel_1866145/1503278224.py:66: RuntimeWarning: Mean of empty slice
  mean_matches.append(np.nanmean(matches))
/tmp/ipykernel_1866145/1503278224.py:67: RuntimeWarning: Mean of empty slice
  mean_num_tokens.append(np.nanmean(num_toks))
/tmp/ipykernel_1866145/1503278224.py:66: RuntimeWarning: Mean of empty slice
  mean_matches.append(np.nanmean(matches))
/tmp/ipykernel_1866145/1503278224.py:67: RuntimeWarning: Mean of empty slice
  mean_num_tokens.append(np.nanmean(num_toks))


random baseline 0.07789028384988256


/tmp/ipykernel_1866145/1503278224.py:66: RuntimeWarning: Mean of empty slice
  mean_matches.append(np.nanmean(matches))
/tmp/ipykernel_1866145/1503278224.py:67: RuntimeWarning: Mean of empty slice
  mean_num_tokens.append(np.nanmean(num_toks))


In [113]:
r.groupby(['checkpoint', 'n_shots'])[['Match accuracy', 'Span length']].mean().round(3)

Match accuracy  Span length
checkpoint       n_shots                             
gpt-4-0314       1                 0.904        2.386
                 5                 0.898        2.263
text-davinci-003 1                 0.822        2.349
                 5                 0.827        2.020